In [1]:
# imports
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep
from datetime import date
import requests
import undetected_chromedriver as uc
from urllib.parse import urljoin,urlencode
import re
import json

In [2]:
res = requests.get('https://www.dice.com/jobs/q-junior+Software+developer-jobs') 
print(res)

<Response [200]>


In [3]:
print(res.text)

<!DOCTYPE html>
<html>









<head>
	



    <style>.async-hide { opacity: 0 !important} </style>
<script>(function(a,s,y,n,c,h,i,d,e){s.className+=' '+y;h.start=1*new Date;
    h.end=i=function(){s.className=s.className.replace(RegExp(' ?'+y),'')};
    (a[n]=a[n]||[]).hide=h;setTimeout(function(){i();h.end=null},c);h.timeout=c;
})(window,document.documentElement,'async-hide','dataLayer',4000,
    {'GTM-WLPMJGK':true});</script>

	







<!-- Google Tag Manager -->
<script>
	(function(w,d,s,l,i) {
		w[l]=w[l]||[];
		w[l].push({'gtm.start':new Date().getTime(),event:'gtm.js'});
		var f=d.getElementsByTagName(s)[0],j=d.createElement(s),dl=l!='dataLayer'?'&l='+l:'';
		j.async=true;j.src='https://www.googletagmanager.com/gtm.js?id='+i+dl;
		f.parentNode.insertBefore(j,f);
	})(window,document,'script','dataLayer','GTM-WLPMJGK');
</script>
<!-- End Google Tag Manager -->

<script src="//assets.dice.com/techpro/js/global/md5.min.js"></script>
<script defer="defer" src="//assets.dice.com/t

In [4]:
def dice_scrape(query = "junior software developer", pages=1, wait=5):
    base = 'https://www.dice.com/jobs/'
    params = {}
    params['q']=query

    titles = []
    companies = []
    links = []
    remote = []

    cities = []
    states= []
    countries=[]
    zips = []

    dates=[]
    descriptions = []

    def scrap_template1(cup):

        description = cup.find('div', id='jobdescSec')
        scripts = cup.find_all('script', type="text/javascript")
        job_info=scripts[4]
        json_txt=re.search(r'"jobId(.|\n)*}',job_info.text)
        json_txt_clean = re.sub(r'//.*', '', json_txt.group())
        json_txt_clean = "{"+json_txt_clean
        job_json = json.loads(json_txt_clean)
        job_json['description']=description.text

        try:
            company = cup.find('span', id='hiringOrganizationName')
            companies.append(company.text)
        except:
            companies.append(None)

        try:
            description = cup.find('div', id='jobdescSec')
            descriptions.append(description.text)
        except:
            descriptions.append(None)

        try:
            scripts = cup.find_all('script', type="text/javascript")
            job_info=scripts[4]
            json_txt=re.search(r'"jobId(.|\n)*}',job_info.text)
            json_txt_clean = re.sub(r'//.*', '', json_txt.group())
            json_txt_clean = "{"+json_txt_clean
            job_json = json.loads(json_txt_clean)
            cities.append(job_json['jobCity'])
            states.append(job_json['jobRegion'])
            countries.append(job_json['jobCountry'])
            zips.append(job_json['jobPostalCode'])
            dates.append(job_json['datePosted'])
            remote.append(job_json['remote'])
        except:
            dates.append(None)
            cities.append(None)
            states.append(None)
            zips.append(None)
            countries.append(None)
            remote.append(None)

    def scrap_template2(cup):
        try:
            description = cup.find('div', id='jobDescription')
            descriptions.append(description.text)
        except:
            descriptions.append(None)

        try:
            company = cup.find('a', attrs={'data-cy': 'companyNameLink'})
            companies.append(company.text)
        except:
            companies.append(None)
            
        try:
            json_text = cup.find('script', id="__NEXT_DATA__")
            jobdic=json.loads(json_text.text)
            jobdata = jobdic['props']['pageProps']['initialState']['api']['queries']
            firstkey = list(jobdata.keys())[0]
            locationdetail = jobdata[firstkey]['data']['locationDetail']
            locationdata = locationdetail['locations'][0]
            dateposted = jobdata[firstkey]['data']['datePosted']
            dates.append(dateposted)
            remote_status = locationdetail['remote']
            remote.append(remote_status)
            cities.append(locationdata['city'])
            countries.append(locationdata['city'])
            states.append(locationdata['state'])
            zips.append(locationdata['postalCode'])

        except:
            dates.append(None)
            cities.append(None)
            states.append(None)
            zips.append(None)
            remote.append(None)
            countries.append(None)

    def scrap_job(joblink):
        res = requests.get(joblink) 
        cup = BeautifulSoup(res.text,'html.parser')

        if cup.find('div', id='jobdescSec'):
            scrap_template1(cup)
        else:
            scrap_template2(cup) 
    # web driver setup
    driver = uc.Chrome()

    for page in range(1,pages+1):
        url = base + urlencode(params).replace('=','-')+f'-jobs?p={page}'
        driver.get(url)
        sleep(wait)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        # parent container for job cards
        jobs_list=soup.find('div', attrs={'id': 'search-results-control'})
        jobs=jobs_list.find_all('h3')
        job_titles = [title.find('span').text for title in jobs]
        job_links = [urljoin(base,job.find('a')['href']) for job in jobs]
        jobs_list.find_all('span',class_='compName')
        titles += job_titles
        links += job_links

    driver.close()
    
    for link in links:
        scrap_job(link)

    df = pd.DataFrame()
    df['Job_Title'] = titles
    df['Company'] = companies
    df['Link'] = links
    df['Description'] = descriptions
    df['Date Posted'] = dates
    df['Country'] = countries
    df['State'] = states
    df['City'] = cities
    df['Zip'] = zips
    df['Remote'] = remote


    return df   
    


In [8]:
df = dice_scrape(pages=5)

In [10]:
df

,Job_Title,Company,Link,Description,Date Posted,Country,State,City,Zip,Remote
0,Junior Software Engineer,Lincoln Financial Group,https://www.dice.com/jobs/detail/Junior-Softwa...,\nAlternate Locations: Work from HomeWork Arra...,2023-02-02T23:26:03Z,[US],[PA],[Radnor],[19087],false
1,Jr Software Developer,Honu Services,https://www.dice.com/jobs/detail/Jr-Software-D...,Location: Nationwide with preference for DC me...,2023-03-19T12:13:16.000Z,None,None,None,None,True
2,"Jr. Application Developer (SQL,...",Randstad Technologies,https://www.dice.com/jobs/detail/Jr.-Applicati...,"job summary:As a Jr. Application Developer, yo...",2023-03-14T16:20:51.000Z,Atlanta,GA,Atlanta,30313,False
3,Jr. Software Developer,"OST, Inc.",https://www.dice.com/jobs/detail/Jr.-Software-...,Description of specific duties in a typical wo...,2023-03-14T14:14:38.000Z,None,None,None,None,True
4,Junior Software Developer,Nigel Frank International,https://www.dice.com/jobs/detail/Junior-Softwa...,Junior Software DeveloperI am currently workin...,2023-03-17T12:10:45.000Z,Arlington,VA,Arlington,22201,False
...,...,...,...,...,...,...,...,...,...,...
95,Senior Software Engineer,Yoh - A Day & Zimmerman Company,https://www.dice.com/jobs/detail/Senior-Softwa...,Senior Software Engineer is needed for a cont...,2023-03-02T19:04:45.000Z,SAN DIEGO,CA,SAN DIEGO,92132,True
96,Software Developer,"iconectiv, LLC.",https://www.dice.com/jobs/detail/Software-Deve...,"\nHybrid work schedule, Bridgewater, NJRespons...",2022-10-13T20:19:49Z,[US],[NJ],[Bridgewater Township],[08807],false
97,Software Developer,Power Home Remodeling,https://www.dice.com/jobs/detail/Software-Deve...,Experienced Software Developer - Release Engin...,2023-03-13T19:23:29.000Z,Philadelphia,PA,Philadelphia,19190,False
98,Sr. Software Engineer-Open Systems...,"G2 Software Systems, Inc.",https://www.dice.com/jobs/detail/Sr.-Software-...,"G2 Software Systems, Inc. (G2) is looking for ...",2022-12-14T00:11:24.000Z,San Diego,CA,San Diego,92101,False


In [9]:
df.to_csv('dice.csv')